In [124]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pandas.tools.plotting import 

In [87]:
dat = pd.read_csv("EEG data.csv")
dat2 = pd.read_csv("demographic info.csv")
dat.shape # There are 12811 observations

(12811, 15)

In [88]:
# Display the first five observations
dat.head()

,subject ID,Video ID,Attention,Meditation,Raw,Delta,Theta,Alpha 1,Alpha 2,Beta 1,Beta 2,Gamma1,Gamma2,predefined label,Self-defined label
0,0,0,56,43,278,301963,90612,33735,23991,27946,45097,33228,8293,0,0
1,0,0,40,35,-50,73787,28083,1439,2240,2746,3687,5293,2740,0,0
2,0,0,47,48,101,758353,383745,201999,62107,36293,130536,57243,25354,0,0
3,0,0,47,57,-5,2012240,129350,61236,17084,11488,62462,49960,33932,0,0
4,0,0,44,53,-8,1005145,354328,37102,88881,45307,99603,44790,29749,0,0


In [95]:
# Drop subject6 and merge
new_dat = dat.merge(dat2, on='subject ID', how='left')
new_dat.shape

(12811, 18)

In [97]:
new_dat = new_dat[new_dat["subject ID"] != 6]
new_dat.shape

(11536, 18)

In [103]:
# Convert to a csv file
new_dat.to_csv("EEG_merged.csv", index = False)

## Now, Do with the new dataset!

In [106]:
dat = pd.read_csv("EEG_merged.csv")
dat.head()

,subject ID,Video ID,Attention,Meditation,Raw,Delta,Theta,Alpha 1,Alpha 2,Beta 1,Beta 2,Gamma1,Gamma2,predefined label,Self-defined label,age,ethnicity,gender
0,0,0,56,43,278,301963,90612,33735,23991,27946,45097,33228,8293,0,0,25,Han Chinese,M
1,0,0,40,35,-50,73787,28083,1439,2240,2746,3687,5293,2740,0,0,25,Han Chinese,M
2,0,0,47,48,101,758353,383745,201999,62107,36293,130536,57243,25354,0,0,25,Han Chinese,M
3,0,0,47,57,-5,2012240,129350,61236,17084,11488,62462,49960,33932,0,0,25,Han Chinese,M
4,0,0,44,53,-8,1005145,354328,37102,88881,45307,99603,44790,29749,0,0,25,Han Chinese,M


In [107]:
# Display the mean of each attribute, grouped by 10 students
mean_dis_stu = dat.groupby(['subject ID']).mean()
mean_dis_stu

,Video ID,Attention,Meditation,Raw,Delta,Theta,Alpha 1,Alpha 2,Beta 1,Beta 2,Gamma1,Gamma2,predefined label,Self-defined label,age
subject ID,,,,,,,,,,,,,,,
0,4.313243,48.630452,48.571768,33.042030,684941.555908,165919.786677,35990.755749,24487.685964,20288.437748,37128.914354,38343.019826,15736.938937,0.473434,0.511499,25.0
1,4.461184,45.561107,57.935434,33.599539,428179.287471,125888.289777,31765.673328,22985.478094,20740.016141,13590.006918,7021.468870,4535.732513,0.489623,0.402767,24.0
2,4.433801,55.971184,60.865265,33.904984,39549.890966,19263.661994,7781.796729,8248.434579,7975.485202,8410.145639,4621.512461,3556.214953,0.482866,0.508567,31.0
3,4.432268,40.044140,45.648402,34.219178,780844.464231,221221.465753,52551.698630,34068.783105,23391.176560,24573.475647,12363.896499,5303.480213,0.384323,0.493912,28.0
4,4.440154,44.895753,52.888803,38.635521,363673.088031,108926.853282,25011.400772,19826.601544,17363.406950,15166.986100,11229.140541,8828.527413,0.487259,0.584556,24.0
5,4.315372,46.616482,54.213946,34.487322,751906.218700,241902.458003,58220.128368,39022.931854,26944.388273,23847.141838,13674.559429,4347.840729,0.473851,0.607765,24.0
7,4.387931,44.844044,50.167712,36.307994,568962.452978,130318.122257,30058.974138,20911.165361,18988.803292,28899.112853,21948.364420,15656.793103,0.479624,0.618339,25.0
8,4.393916,43.990640,50.177847,35.666147,830712.153666,236931.875195,54706.961778,38513.213729,26461.799532,24249.097504,15605.233229,9436.329173,0.482059,0.402496,25.0
9,4.313243,42.507534,51.110230,29.070579,870734.973830,180547.519429,51447.114195,45816.708168,20216.413957,46573.854084,28369.115781,6730.126090,0.473434,0.498810,24.0


In [24]:
dat.describe()

,subject ID,Video ID,Attention,Meditation,Raw,Delta,Theta,Alpha 1,Alpha 2,Beta 1,Beta 2,Gamma1,Gamma2,predefined label,Self-defined label
count,12811.000000,12811.000000,12811.000000,12811.000000,12811.000000,1.281100e+04,1.281100e+04,1.281100e+04,1.281100e+04,1.281100e+04,1.281100e+04,1.281100e+04,1.281100e+04,12811.000000,12811.000000
mean,4.487394,4.390602,41.313871,47.182656,65.570760,6.057853e+05,1.680526e+05,4.138435e+04,3.318339e+04,2.431837e+04,3.814433e+04,2.959255e+04,1.441597e+04,0.470377,0.512606
std,2.865373,2.913232,23.152953,22.655976,597.921035,6.376236e+05,2.441346e+05,7.243082e+04,5.831410e+04,3.837968e+04,7.906606e+04,7.982637e+04,3.603523e+04,0.499141,0.499861
min,0.000000,0.000000,0.000000,0.000000,-2048.000000,4.480000e+02,1.700000e+01,2.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00,1.000000e+00,2.000000e+00,0.000000,0.000000
25%,2.000000,2.000000,27.000000,37.000000,-14.000000,9.806400e+04,2.691750e+04,6.838000e+03,6.852000e+03,6.140000e+03,7.358500e+03,4.058000e+03,2.167500e+03,0.000000,0.000000
50%,4.000000,4.000000,43.000000,51.000000,35.000000,3.954870e+05,8.133100e+04,1.750000e+04,1.495900e+04,1.281800e+04,1.581000e+04,9.763000e+03,5.116000e+03,0.000000,1.000000
75%,7.000000,7.000000,57.000000,63.000000,90.000000,9.166230e+05,2.052760e+05,4.477950e+04,3.455050e+04,2.740600e+04,3.549400e+04,2.488800e+04,1.266950e+04,1.000000,1.000000
max,9.000000,9.000000,100.000000,100.000000,2047.000000,3.964663e+06,3.007802e+06,1.369955e+06,1.016913e+06,1.067778e+06,1.645369e+06,1.972506e+06,1.348117e+06,1.000000,1.000000


In [132]:
q = dat.corr()
names <- dat.columns
# plot correlation matrix
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(q, vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,9,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(names)
ax.set_yticklabels(names)
plt.show()

NameError: name 'names' is not defined

In [108]:
count_per_subject_id = dat.groupby(['subject ID']).size()
count_per_subject_id # Oh well.. the number of observations per students are not the same!

subject ID
0    1261
1    1301
2    1284
3    1314
4    1295
5    1262
7    1276
8    1282
9    1261
dtype: int64

In [ ]:
# Remove subject ID 6.
# gender, age, ethnicity.. combine data

In [ ]:
# Some tasks to be considered
# 1. Since we have both predefined and user-defined labels, simply compare them to calculate accuracy (the most boring task)
# 2. Use usef-defined label as a target variable and build a prediction
# 3. Without knowing their confusion level, create clusters? 
# 4. etc...

# We could use Logistic Regression / Decision Tree / Naive Baye / SVM / etc.... 
# We can also use some Ensemble learning methods... to be considered

In [134]:
1261 * 9

11349